# Описание

Исходный репозиторий: https://github.com/RobZuazua/kiwi-mental-health-chatbot

Логика работы:

Использует базу http://www.nacbt.org/whatiscbt-htm/ для диалога, но самих данных в репозитории нет.

Для логики использует внешнее API dialoflow (https://cloud.google.com/dialogflow/docs/), пакет apiai
Примеры работы с apiai:
* Погодный чат бот телеграмм https://habr.com/ru/post/336668/
* голосовой бот https://habr.com/ru/post/515128/
* настройки интеграции с carrot https://help.carrotquest.io/article/139
* новая версия (на примере энтепрайз) https://medium.com/google-cloud/how-to-create-a-chatbot-using-dialogflow-enterprise-edition-and-dialogflow-api-v2-923f4a965176
* работа с диалог флоу в питон https://medium.com/swlh/working-with-dialogflow-using-python-client-cb2196d579a4

Сервис стал платный. Условия https://cloud.google.com/dialogflow/docs/editions#es-agent Есть базовая демо версия

Для обработки текста (NLP) используется spacy https://spacy.io/

В СУБД хранятся сессии и результаты беседы.

Flask как веб оболочка

## Предварительные действия

Регистрируем агента для apiai:
1. Переходим https://dialogflow.cloud.google.com/#/login
2. Авторизуемся
3. Нажимаем создать агента
4. Задаем имя агента и язык, например `ru`
5. Создаем и переходим по ссылки в созданный гугл проект. Запоминаем название проекта, например `test-xhbs` и записываем его в DIALOGFLOW_PROJECT_ID
6. Переходим в настройки проекта и раздел `Service accoun`
7. Создаем сервисный аккаунт
8. Выбираем роль `Dialogflow API Client` и `Done`
9. Выбираем для аккаунта свойства (три точки) и `Manage keys`
10. Создаем ключ и сохраняем json с названием `dialogflow-test-key.json`
11. Кладем ключ в папку на диске `/content/gdrive/MyDrive/chats_emotions_and_voises/chat03_kiwi-mental-health-chatbot`. Путь указан в os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
12. Переходим в Intents dialogflow и кликаем на добавить.
13. Вводим название `Worry`
14. В `Training phrases` добавляем фразы на которые будут реагировать как на беспокойство, для примера `беспокоит` и `раздражает` - на эти ключевые слова будет активирован сценарий `Worry` и включе дополнительный анализ в программе. Можно восстановитьи из архива `dialogflow_kiwi-mental-health-chatbot.zip`
15. В `Responses` добавляем типовой ответ, например `Что вас беспокоит`
16. В последней ячейке с формой вводим текст для запроса, например `если телевизор меня беспокоит тогда я хочу его выключить`

Ограничения и особенности модели:
* Ключевые слова для связанных текстов `если`, `тогда`, `потому` на них реагирует этим модулем и заменяет ответ dialogflow на построенный в функции `checkRelationships`
* Функция наводящих вопросв на предмет не работает, так как у scapy для русского языка функционал парсинга по местоположению слов в предложении не реализован из-за возможности нахождения ключевых слов в произвольном месте предложения.



In [14]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/gdrive/MyDrive/data/dialogflow-test-key.json' #'/content/gdrive/MyDrive/chats_emotions_and_voises/chat03_kiwi-mental-health-chatbot/dialogflow-test-key.json'

DIALOGFLOW_PROJECT_ID = 'test-xhbs'
DIALOGFLOW_LANGUAGE_CODE = 'ru'
SESSION_ID = 'me'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Устанавливаем зависимости. После нужно перезапустить ядро!
!pip install simplejson google-cloud-dialogflow==2.1.2 spacy==3.0.6 #download
!pip install dialogflow google-api-core

     |████████████████████████████████| 358kB 4.0MB/s 


In [1]:
!python -m spacy download ru_core_news_sm

2021-06-04 17:21:05.561079: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [4]:
import spacy
import simplejson as json
import dialogflow

from google.api_core.exceptions import InvalidArgument

In [5]:
# Функция инициализации общения (убрано сохранение )
def createStateObjAndCookie(cookie,stateObj):
        stateObj['id']=cookie['value']
        stateObj['worry']=''
        stateObj['relationshipA']=''
        stateObj['relationshipB']=''
        stateObj['certainty']=''
        stateObj['last']=''
        stateObj['lastResponse']=''

In [7]:
def updateState(stateObj):
        if not stateObj['worry']:
          stateObj['worry'] = 'NULL'
        else:
          stateObj['worry'] = "\'"+stateObj['worry']+'\''
        if not stateObj['relationshipA']:
          stateObj['relationshipA'] = 'NULL'
        else:
          stateObj['relationshipA'] = "\'"+stateObj['relationshipA']+'\''
        if not stateObj['relationshipB']:
          stateObj['relationshipB'] = 'NULL'
        else:
          stateObj['relationshipB'] = "\'"+stateObj['relationshipB']+'\''
        if not stateObj['certainty']:
          stateObj['certainty'] = 'NULL'
        else:
          stateObj['certainty'] = "\'"+stateObj['certainty']+'\''
        if not stateObj['last']:
          stateObj['last'] = 'NULL'
        else:
          stateObj['last'] = "\'"+stateObj['last']+'\''
        if not stateObj['lastResponse']:
          stateObj['lastResponse'] = 'NULL'
        else:
          stateObj['lastResponse'] = "\'"+stateObj['lastResponse']+'\''

        return

In [19]:
# Переделано на новую версию API

def dialogflow_requrst(message):  
    text_to_be_analyzed = message
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=text_to_be_analyzed, language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
        raise

    print("ВОПРОС:", response.query_result.query_text)
    print("        Detected intent:", response.query_result.intent.display_name)
    print("        Detected intent confidence:", response.query_result.intent_detection_confidence)
    print("        Fulfillment text:", response.query_result.fulfillment_text)

    return response.query_result


In [25]:
def spacyHandleWorries(stateObj, message):
    # nlp = spacy.load('en')

    nlp = spacy.load("ru_core_news_sm")
    # nlp = Russian()     #nlp = spacy.load('ru')
    doc = nlp(message)
    noun_chunks = {}
    try:
      for chunk in doc.noun_chunks: # Определяет связи слов в предложении, не применимо для русского языка и выбрасывает ошибку
          noun_chunks.update({chunk.root.dep_:chunk})
    except Exception as err:
      print(err)

    found = ''
    dependencies = list(noun_chunks.keys()) # Схема объектов https://spacy.io/usage/spacy-101
    if 'dobj' in dependencies: 
        found = noun_chunks['dobj']
    elif 'pobj' in dependencies:
        found = noun_chunks['pobj']
    elif 'pcomp' in dependencies:
        found = noun_chunks['pcomp']

    if found:
        #stateObj['worry'].append(found)
        stateObj['worry'] = str(found)
        stateObj['last'] = "worry"
        # stateObj['lastResponse'] = "Why are you worried about {}?".format(found)
        stateObj['lastResponse'] = "Почему вы об {} беспокоитесь?".format(found)
    else:
        #stateObj['lastResponse'] = "I understand you are worried about something, what is it that you are worried about?"
        stateObj['lastResponse'] = "Я понимаю что вы о чем-то беспокоитесь. Но почему вас это беспокоит?"
        stateObj['last'] = "worry"

def checkCertainties(stateObj,message):
    # nlp = spacy.load('en')
    nlp = spacy.load("ru_core_news_sm")
    doc = nlp(message)
    tokens = []
    for token in doc:
        tokens.append(str(token).lower())
    print('   Токены', tokens)
    certainWords = ["думаю", "считаю", "уверен"]
    foundCertainWords = list(set(tokens).intersection(certainWords))
    print('    Найденные слова уверенности', foundCertainWords)    

    if foundCertainWords:
        if len(foundCertainWords) == 1:
            certainWord = foundCertainWords[0]
            certainWord_index = tokens.index(certainWord)
            certainty = ' '.join(tokens[certainWord_index+1:])
            #stateObj['certainty'].append(certainty)
            stateObj['certainty']=certainty
            stateObj['last'] = 'certainty'
            # stateObj['lastResponse'] = 'You seem pretty certain that {}. What is the reasoning behind this?'.format(certainty)
            stateObj['lastResponse'] = 'Вы выглядите достаточно уверенными {}. Что стоит за этим?'.format(certainty)
            return
        else:
            stateObj['last']='certainty'
            # stateObj['lastResponse']='Tell me more about why you are certain about this'
            stateObj['lastResponse']='Расскажите мне почему вы так уверены в этом'
            return
    # stateObj['lastResponse='] = 'Tell me about what is on your mind'
    stateObj['lastResponse='] = 'Расскажите мне, что у вас на уме'
    return

In [10]:
def checkRelationships(stateObj, message):
    # nlp = spacy.load('en')
    nlp = spacy.load("ru_core_news_sm")
    doc = nlp(message)
    tokens = []
    print('doc is' ,doc)
    for token in doc:
        tokens.append(str(token).lower())
    #relationshipWords = ["if", "then", "because"]
    relationshipWords = ["если", "тогда", "потому"]
    foundRelationshipWords = set(tokens).intersection(relationshipWords)


    if foundRelationshipWords:
        if "если" in foundRelationshipWords and "тогда" in foundRelationshipWords:
            if_index = tokens.index("если")
            then_index = tokens.index("тогда")
            if if_index < then_index:
                a = ' '.join(tokens[if_index+1:then_index])
                b = ' '.join(tokens[then_index+1:])
                stateObj['relationshipA']=a
                stateObj['relationshipB']=b
                print(stateObj['relationshipA'])
                #stateObj['relationships'].append([a,b])
                stateObj['last'] = 'relationship'
                # stateObj['lastResponse'] = 'I understand you believe {} implies {}. Maybe we can talk more about this. What do you think the consequence of this is?'.format(a,b)
                stateObj['lastResponse'] = 'Я понимаю, что вы веря в {} подразумеваете {}. Может быть мы можем поговорить об этом. Какие вы думаете у этого последствия?'.format(a,b)
                return
            else:
                stateObj['last']=''
                # stateObj['lastResponse']='That sounded pretty complex. Tell me more about your reasoning'
                stateObj['lastResponse']='Это звучит достаточно сложно. Расскажите мне о ваших рассуждениях'
                return

        if "если" in foundRelationshipWords:
            if_index = tokens.index("если")
            if if_index:
                a = ' '.join(tokens[if_index+1:])
                b = ' '.join(tokens[:if_index])
                #stateObj.relationships.append([a,b])
                stateObj['relationshipA']=a
                stateObj['relationshipB']=b
                stateObj['last'] = 'relationship'
                # stateObj['lastResponse'] = 'I understand you believe {} implies {}. Maybe we can talk more about this. What do you think the consequence of this is?'.format(a,b)
                stateObj['lastResponse'] = 'Я понимаю, что вы говоря о {} подразумеваете {}. Может быть мы можем поговорить об этом. Какие вы думаете у этого последствия?'.format(a,b)
                return
            else:
                stateObj['last']=''
                # stateObj['lastResponse']='Hmm.. Tell me more about your reasoning'
                stateObj['lastResponse']='Хм... Расскажите мне о ваших рассуждениях'
                return


        if "потому" in foundRelationshipWords:
            because_index = tokens.index("потому")
            if because_index:
                a = ' '.join(tokens[because_index+1:])
                b = ' '.join(tokens[:because_index])
                stateObj['relationshipA']=a
                stateObj['relationshipB']=b
                stateObj['last'] = 'relationship'
                # stateObj['lastResponse'] = 'I understand you believe {} implies {}. Maybe we can talk more about this. What do you think the consequence of this is?'.format(a,b)
                stateObj['lastResponse'] = 'Я понимаю, что вы веря в {} подразумеваете {}. Может быть мы можем поговорить об этом. Какие вы думаете у этого последствия?'.format(a,b)
                return
            else:
                stateObj['last']=''
                # stateObj['lastResponse']='I see, what is the consequence of that?'
                stateObj['lastResponse']='Понимаю. А какие последствия у этого?'
                return

        # stateObj['lastResponse']='Tell me about your feelings'
        stateObj['lastResponse']='Расскажите мне о том, что вы чувствуете'
        return

In [11]:
def matchIntent(stateObj, dialogflowResponse, userMessage):

    # intent = dialogflowResponse['result']['metadata']['intentName']
    intent = dialogflowResponse.intent.display_name
    # responseDialog = dialogflowResponse['result']['fulfillment']['messages'][0]['speech']
    responseDialog = dialogflowResponse.fulfillment_text

    stateObj['last']=''

    # if intent == 'Default Welcome Intent' or intent == 'not sure':
    if intent == 'Default Welcome Intent' or intent == 'Default Fallback Intent':
        return responseDialog

    if intent == 'Worry':
        spacyHandleWorries(stateObj, userMessage)

    checkRelationships(stateObj, userMessage)

    if not stateObj['relationshipA'] or stateObj['relationshipA'] == 'NULL':
        checkCertainties(stateObj,userMessage)

    updateState(stateObj)

    if not stateObj['lastResponse'] or stateObj['lastResponse'] == 'NULL':
      stateObj['lastResponse'] = responseDialog

    if stateObj['lastResponse'][0] == "'":
      stateObj['lastResponse'] = stateObj['lastResponse'][1:-1]

    stateObj['lastResponse']=stateObj['lastResponse'].replace('мой','ваш')
    print("ЗДЕСЬ РЕЗУЛЬТАТ: " + stateObj['lastResponse'])

    return stateObj['lastResponse']

In [17]:
# Входная функция получения сообщения
def apiEndpoint(data):
    """
    data: {userMessage: message}
    """
    # data = request.get_json()
    stateObj={}
    cookie = {'value':'123'}
    createStateObjAndCookie(cookie, stateObj) # Убарны куки

    print(stateObj)
    userMessage = data['userMessage']
    # dialogflowResponse = dialogflow(userMessage); # Нельзя запрос называть как библиотека
    dialogflowResponse = dialogflow_requrst(userMessage)
    
    # print(dialogflowResponse)
    apiResponse = matchIntent(stateObj,dialogflowResponse,userMessage)
    print('\nBOT:', apiResponse)

In [27]:
#@title Default title text
#dialog_str1 = "\u041A\u0430\u043A \u0434\u0435\u043B\u0430?" #@param {type:"string"}
while(True):
  dialog_str1 = input('USER: ')
  apiEndpoint({'userMessage': dialog_str1})
# print(dialog_str1)


USER: Привет
{'id': '123', 'worry': '', 'relationshipA': '', 'relationshipB': '', 'certainty': '', 'last': '', 'lastResponse': ''}
ВОПРОС: Привет
        Detected intent: Default Welcome Intent
        Detected intent confidence: 1.0
        Fulfillment text: Здравствуй!

BOT: Здравствуй!
USER: Как дела?
{'id': '123', 'worry': '', 'relationshipA': '', 'relationshipB': '', 'certainty': '', 'last': '', 'lastResponse': ''}
ВОПРОС: Как дела?
        Detected intent: Default Fallback Intent
        Detected intent confidence: 1.0
        Fulfillment text: Вот сейчас я тебя совсем не понимаю.

BOT: Вот сейчас я тебя совсем не понимаю.
USER: Это то и раздражает
{'id': '123', 'worry': '', 'relationshipA': '', 'relationshipB': '', 'certainty': '', 'last': '', 'lastResponse': ''}
ВОПРОС: Это то и раздражает
        Detected intent: Worry
        Detected intent confidence: 0.7000908851623535
        Fulfillment text: Что вас беспокоит?
[E894] The 'noun_chunks' syntax iterator is not implemented 

KeyboardInterrupt: ignored